In [1]:
from __future__ import division, print_function, absolute_import

In [2]:
import time
from collections import OrderedDict

import tensorflow as tf

import pandas as pd

import numpy as np

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
cols = ['Ref', 'RefComposite', 'RhoHV', 'Zdr', 'Kdp']

df_raw = pd.read_csv('df_train.csv').set_index('Id')[cols+['minutes_past','Expected']]
ixs = np.array(df_raw.index.unique())
np.random.shuffle(ixs)
pivot = int(0.9*len(ixs))  # train_test splitting pivot
train_ixs, test_ixs = ixs[:pivot], ixs[pivot:]

df_train_raw = df_raw.loc[train_ixs,:]
df_test_raw = df_raw.loc[test_ixs,:]

df_train = df_train_raw.drop('minutes_past',1)
df_train.sample(5)

,Ref,RefComposite,RhoHV,Zdr,Kdp,Expected
Id,,,,,,
4565.0,30.500000,30.50,0.974524,1.678571,1.349996,2.032001
4043.0,16.000000,16.00,0.976558,0.604019,-0.027961,0.508000
4042.0,26.500000,26.50,0.976558,0.604019,-0.027961,2.286001
2577.0,28.000000,30.00,1.001667,-2.187500,0.000000,0.020000
252.0,11.611111,11.95,0.976558,0.604019,-0.027961,0.254000


In [5]:
df_test = df_test_raw.drop('minutes_past',1)
df_test.head(5)

,Ref,RefComposite,RhoHV,Zdr,Kdp,Expected
Id,,,,,,
1659.0,23.431635,25.146985,0.976558,0.604019,-0.027961,82.04204
1659.0,23.431635,25.146985,0.976558,0.604019,-0.027961,82.04204
1659.0,23.431635,25.146985,0.976558,0.604019,-0.027961,82.04204
1659.0,23.431635,25.146985,0.976558,0.604019,-0.027961,82.04204
1659.0,23.431635,25.146985,0.976558,0.604019,-0.027961,82.04204


### Scaling the data

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()
train_X = np.array(df_train.drop('Expected',1))
train_y = np.array(df_train['Expected'])

test_X = np.array(df_test.drop('Expected',1))
test_y = np.array(df_test['Expected'])

scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)

In [8]:
df_train_scaled = pd.DataFrame(train_X,columns=cols, index=df_train.index)
df_train_scaled['Expected'] = df_train.Expected

df_test_scaled = pd.DataFrame(test_X,columns=cols, index=df_test.index)
df_test_scaled['Expected'] = df_test.Expected

### Generating batches

In [9]:
def next_id(df):
    ids = sorted(df.index.unique())
    while True:
        for i in ids:
            yield i

In [10]:
def df2instance(df, num_steps, labled):
    df = df.head(num_steps)
    instance = np.array(df)
    padding_width = num_steps-len(instance)
    instance = np.pad(instance, (0,padding_width), 'constant')
    if padding_width:
        instance = instance[:,:-padding_width]
    if labled:
        x, y = instance[:,:-1], instance[:,-1]
        return x,y
    else:
        return instance

In [11]:
def next_batch(df, batch_size=100, num_steps=10, labled=True):
    id_generator = next_id(df)
    while True:
        counter = 0
        seq_lens = list()
        batch_x, batch_y = list(), list()
        while counter<batch_size:
            id_ = id_generator.__next__()        
            batch_df = df.loc[id_]
            seq_lens.append(len(batch_df))
            counter += len(batch_df)
            
            x, y = df2instance(batch_df, num_steps, labled)
            batch_x.append(x)
            batch_y.append(y)
            
        yield np.array(batch_x), np.array(batch_y), np.array(seq_lens)

### Defining device cell wrappers

In [12]:
class DeviceCellWrapper(tf.contrib.rnn.RNNCell):
    def __init__(self, device, cell):
        self._cell = cell
        self._device = device

    @property
    def state_size(self):
        return self._cell.state_size

    @property
    def output_size(self):
        return self._cell.output_size

    def __call__(self, inputs, state, scope=None):
        with tf.device(self._device):
            return self._cell(inputs, state, scope)

## Deep RNN
<img src="misc/deeprnn.png">

## Using traditional regression models

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error

In [14]:
X = np.array(df_train_scaled.drop('Expected', 1))
y = np.array(df_train_scaled.Expected)

X_test = np.array(df_test_scaled.drop('Expected', 1))
y_test = np.array(df_test_scaled.Expected)

In [15]:
models = {"Linear Regression":LinearRegression,
          "RandomForest Regressor": RandomForestRegressor,
          "SVR":SVR}

In [16]:
for name,model in models.items():
    start = time.time()
    if 'SVR' in name:
        m = model(max_iter=1000)
    else:
        m = model(n_jobs=-1)
    m.fit(X,y)
    preds = m.predict(X_test)
    train_preds = m.predict(X)
    print("{0}:\nTrain MAE: {1}\nTest MAE: {2}\nTook: {3} seconds".format(
        name,
        mean_absolute_error(y, train_preds),
        mean_absolute_error(y_test, preds),
        time.time()-start))
    print()

RandomForest Regressor:
Train MAE: 198.616397488946
Test MAE: 187.7760663359275
Took: 0.5199010372161865 seconds



/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVR:
Train MAE: 1393.8859559139942
Test MAE: 1393.0591196762025
Took: 9.539725542068481 seconds

Linear Regression:
Train MAE: 238.78614226156066
Test MAE: 237.78300350773745
Took: 0.005511283874511719 seconds



## Using 2 GRU layers
<img src='misc/gru.png'>

In [24]:
num_inputs = len(df_test.columns)-1
num_outputs = 1
num_units = 100
num_layers = 2
num_epochs = 100
batch_size=256

num_steps = int(df_train.groupby(df_train.index).count().max()[0])
num_steps

19

In [25]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('inputs'):
        X = tf.placeholder(tf.float32, [None, num_steps, num_inputs])
        Y = tf.placeholder(tf.float32, [None, num_steps, num_outputs])
        seq_lens = tf.placeholder(tf.int32, [None])
        keep_prob = tf.placeholder_with_default(1.0, shape=(), name='training')  # 0.5 in training, 1.0 in testing
        
    with tf.name_scope('gru_cells'):
        gru_cells = [tf.contrib.rnn.GRUCell(num_units) for i in range(num_layers)]
        
        gru_cells = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=keep_prob)
                      for cell in gru_cells]
        deep_cell = tf.contrib.rnn.MultiRNNCell(gru_cells)
        
    with tf.name_scope("outputs"):
        outputs, states = tf.nn.dynamic_rnn(deep_cell, X, sequence_length=seq_lens, dtype=tf.float32)
        
        # Output projection
        stacked_rnn_outputs = tf.reshape(outputs, (-1, num_units))
        stacked_outputs = tf.contrib.layers.fully_connected(stacked_rnn_outputs, num_outputs, activation_fn=None)
        outputs = tf.reshape(stacked_outputs, (-1, num_steps, num_outputs))
    
    with tf.name_scope("optimization"):
        loss = tf.reduce_mean(tf.losses.absolute_difference(Y, outputs))
        optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(loss)
        
    _saver = tf.train.Saver()
    init = tf.global_variables_initializer()    

In [26]:
g = next_batch(df_test_scaled, batch_size=len(df_test_scaled), num_steps=num_steps)
test_X, test_Y, test_seq_lens = g.__next__()
del g

In [27]:
batch_generator = next_batch(df_train_scaled, batch_size=batch_size, num_steps=num_steps)
batch_X, batch_Y, batch_seq_lens = batch_generator.__next__()

In [20]:
with tf.Session(graph=graph) as sess:
    with tf.device('/gpu'):
        init.run()
        batch_generator = next_batch(df_train_scaled, batch_size=batch_size, num_steps=num_steps)
        train_losses = OrderedDict()
        test_losses = OrderedDict()
        for e in range(num_epochs+1):
            avg_mae = 0
            for i in range(len(df_train)//batch_size):
                batch_X, batch_Y, batch_seq_lens = batch_generator.__next__()
                _, mae = sess.run([optimizer, loss],
                                  feed_dict={X:batch_X,
                                             Y:batch_Y.reshape(-1, num_steps, num_outputs),
                                             seq_lens: batch_seq_lens,
                                             keep_prob:0.5})
                
                avg_mae += mae
            train_losses[e] = avg_mae
            test_mae = sess.run([loss],
                                feed_dict={X:test_X,
                                           Y:test_Y.reshape(-1, num_steps, num_outputs),
                                           seq_lens:test_seq_lens,
                                           keep_prob:1.0})[0]
            test_losses[e] = test_mae
            print("Epoch {0}:\nTrain MAE: {1}\nTest MAE: {2}".format(e, avg_mae, test_mae))
            print("~"*30)
        test_preds = sess.run(outputs, {X:test_X,
                                           Y:test_Y.reshape(-1, num_steps, num_outputs),
                                           seq_lens:test_seq_lens,
                                           keep_prob:1.0})
        _saver.save(sess, './rain.s')

Epoch 0:
Train MAE: 15513.328709244728
Test MAE: 72.33020782470703
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 1:
Train MAE: 15468.85962086916
Test MAE: 72.30280303955078
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch 2:
Train MAE: 15408.797367572784
Test MAE: 72.27076721191406
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


KeyboardInterrupt: 

### Plotting results

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,7))
xticks = sorted(train_losses.keys())
data1 = [train_losses[i] for i in xticks]
data2 = [test_losses[i] for i in xticks]

color = 'tab:blue'
ax1.set_xlabel('Epochs')
ax1.set_ylabel('train MAE', color=color)
ax1.plot(xticks, data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('test MAE', color=color)
ax2.plot(xticks, data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped


In [ ]:
x_ticks = df_test_raw.minutes_past/100+df_test_raw.index

In [ ]:
indexed = df_test_raw.reset_index()

In [ ]:
test_lables = np.concatenate([test_Y[i,:test_seq_lens[i]] for i in test_seq_lens])

In [ ]:
preds = np.concatenate(
    [test_preds[i,:test_seq_lens[i],:] for i in test_seq_lens]).reshape(-1)

In [ ]:
# x = np.array(indexed.index)
# y = np.array(indexed.Expected)
x = np.arange(len(test_lables))
plt.figure(figsize=(20,7))
plt.xticks = x_ticks
plt.plot(x[:100], test_lables[:100])
plt.plot(x[:100], preds[:100])
plt.show()